In [ ]:
import sys

import open3d as o3d

# Python version
assert sys.version_info >= (3, 8)
# Open3D version
assert o3d.__version__ >= "0.17.0"

In [ ]:
# File directories
input_path = "data/input/"
preprocessed_path = "data/preprocessed/"
transformed_path = "data/transformed/"
output_path = "data/output/"

i_include = list(range(1, 11))

In [ ]:
import subprocess
import pty
import os


# Run a command and print its output in real-time.
def run_command_realtime(command):
    # Create a pseudo-terminal and spawn the subprocess
    master, slave = pty.openpty()
    process = subprocess.Popen(command, stdin=slave, stdout=slave, stderr=slave, universal_newlines=True)

    # Close the slave file descriptor as it's not needed
    os.close(slave)

    # Read and print the output line by line in real-time
    while True:
        output_line = os.read(master, 1024).decode()
        if not output_line:
            break
        print(output_line, end='', flush=True)

    # Wait for the process to finish
    process.wait()

# Part 1. Point Cloud Processing

## Step 1. Preprocess

In [ ]:
# List of file paths to process
file_paths = [os.path.join(input_path, f"iScan-Pcd-1-{i}.ply") for i in i_include]

# Define the command as a list of strings
command_to_run = ["python", "preprocess.py"]
command_to_run.extend(file_paths)

# Run the command
run_command_realtime(command_to_run)

## Step 2. Transform

In [ ]:
# List of file paths to process
file_paths = [os.path.join(preprocessed_path, f"iScan-Pcd-1-{i} - preprocessed.ply") for i in i_include]

# Define the command as a list of strings
command_to_run = ["python", "transform.py"]
command_to_run.extend(file_paths)

# Run the command
run_command_realtime(command_to_run)

## Step 3. Visualized analysis

In [ ]:
# List of file paths to process
file_paths = [os.path.join(transformed_path, f"iScan-Pcd-1-{i} - transformed.ply") for i in i_include]

# Define the command as a list of strings
command_to_run = ["python", "visualized_analysis.py"]
command_to_run.extend(file_paths)

# Run the command
run_command_realtime(command_to_run)

# Part 2. Plot analysis results

In [ ]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt

# Load the JSON data
json_file_path = os.path.join("data/", "analysis_results.json")
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Extract data for plotting
slice_data = []
for file_info in data["files"]:
    for slice_info in file_info["slices"]:
        slice_data.append({
            "slice_number": slice_info["slice_number"],
            "left_slope": slice_info["left_slope"],
            "width_of_top_surface": slice_info["width_of_top_surface"]
        })
    break
    
slice_numbers = [item["slice_number"] for item in slice_data]
left_slopes = [item["left_slope"] for item in slice_data]
width_of_top_surface = [item["width_of_top_surface"] for item in slice_data]

# Create two subplots (one for left_slope, one for width_of_top_surface)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Plot for left_slope
ax1.plot(slice_numbers, left_slopes, label="Left Slope", marker='s')
ideal_left_slope = 1.75
average_left_slope = np.mean(left_slopes)
ax1.axhline(ideal_left_slope, color="red", linestyle="--", label="Ideal Slope")
ax1.axhline(average_left_slope, color="limegreen", linestyle="--", label="Average Left Slope")
ax1.set_ylim(min(min(left_slopes), ideal_left_slope) - 0.1, max(max(left_slopes), ideal_left_slope) + 0.1)
ax1.set_xlabel("Slice Number")
ax1.set_ylabel("Left Slope")
ax1.set_title("Variation of Left Slope")
ax1.legend()

# Add text annotations for left_slope
for x, y in zip(slice_numbers, left_slopes):
    ax1.annotate(f"{y:.2f}", xy=(x, y), textcoords="offset points",
                 xytext=(0, 10 if y >= average_left_slope else -20),
                 ha="center", fontsize=8, color="blue")

# Plot for width_of_top_surface
ax2.plot(slice_numbers, width_of_top_surface, label="Width of Top Surface", marker='s')
ideal_width = 3.1
average_width = np.mean(width_of_top_surface)
ax2.axhline(ideal_width, color="red", linestyle="--", label="Ideal Width")
ax2.axhline(average_width, color="limegreen", linestyle="--", label="Average Width")
ax2.set_ylim(min(min(width_of_top_surface), ideal_width) - 0.02, max(max(width_of_top_surface), ideal_width) + 0.02)
ax2.set_xlabel("Slice Number")
ax2.set_ylabel("Width of Top Surface (m)")
ax2.set_title("Variation of Width of Top Surface")
ax2.legend()

# Add text annotations for width_of_top_surface
for x, y in zip(slice_numbers, width_of_top_surface):
    ax2.annotate(f"{y:.2f}", xy=(x, y), textcoords="offset points",
                 xytext=(0, 10 if y >= average_width else -20),
                 ha="center", fontsize=8, color="blue")

# Adjust layout and display the plots
plt.tight_layout()
plt.show()